In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00


In [13]:
import re
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import load_dataset

In [17]:

def train_model():

    # Загружаем датасет из текстового файла
    dataset = load_dataset("text", data_files={"train": "train.txt"}, split="train")

    # Загружаем предобученный токенизатор BERT
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Функция для токенизации текстовых данных
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

    # Применяем токенизацию к датасету
    tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

    # Загружаем модель для masked language modeling
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")

    # Создаем data collator для случайного маскирования токенов (15% токенов)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    # Определяем аргументы обучения
    training_args = TrainingArguments(
        output_dir="./bert_mlm",
        overwrite_output_dir=True,
        num_train_epochs=1,
        per_device_train_batch_size=16,
        save_steps=10000,
        save_total_limit=2,
        prediction_loss_only=True,
    )

    # Создаем объект Trainer для обучения модели
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    # Запускаем процесс обучения
    trainer.train()

    # Сохраняем модель и токенизатор
    trainer.save_model("./bert_mlm")
    tokenizer.save_pretrained("./bert_mlm")

    return model, tokenizer

In [15]:
def fill_placeholders(text, fill_mask, tokenizer):
    """
    Функция ищет шаблоны вида *[число]* и последовательно заменяет их
    на предсказанные моделью слова.
    """
    # Регулярное выражение для поиска шаблона, например: *[0]*, *[1]* и т.д.
    pattern = r"\*\[\d+\]\*"

    # Пока в тексте встречается хотя бы один шаблон
    while re.search(pattern, text):
        # Заменяем первое найденное вхождение на токен [MASK]
        text = re.sub(pattern, tokenizer.mask_token, text, count=1)
        # Получаем предсказание для токена [MASK]
        predictions = fill_mask(text)
        # Выбираем топовое предсказание (можно расширить, если нужно учитывать несколько вариантов)
        predicted_token = predictions[0]['token_str'].strip()
        # Заменяем первый найденный [MASK] на предсказанное слово
        text = text.replace(tokenizer.mask_token, predicted_token, 1)
    return text

In [18]:
def validate_model(model, tokenizer):
    """
    Функция для валидации: загружает test.txt, заполняет в нем шаблоны
    и выводит итоговый текст.
    """
    # Загружаем текст для валидации
    with open("test.txt", "r", encoding="utf-8") as f:
        test_text = f.read()

    # Создаем pipeline для задачи fill-mask
    fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

    # Заполняем все шаблоны в тексте
    filled_text = fill_placeholders(test_text, fill_mask, tokenizer)

    print("Заполненный текст:\n")
    print(filled_text)

def main():
    # Обучаем модель на train.txt
    model, tokenizer = train_model()
    # Валидируем модель на test.txt
    validate_model(model, tokenizer)

if __name__ == "__main__":
    main()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss


Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (15707 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (15707) must match the size of tensor b (512) at non-singleton dimension 1

In [21]:
import re
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import load_dataset

def train_model():
    """
    Функция для обучения модели BERT на задаче Masked Language Modeling.
    Использует train.txt как обучающий датасет.
    """
    # Загружаем датасет из текстового файла
    dataset = load_dataset("text", data_files={"train": "train.txt"}, split="train")

    # Загружаем предобученный токенизатор BERT
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Функция для токенизации и разбиения длинных текстов на части
    def tokenize_function(examples):
        # Токенизируем с добавлением параметра return_overflowing_tokens=True
        # для разбиения длинных текстов на части по 512 токенов
        return tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=512,
            stride=128,  # Перекрытие между частями текста
            return_overflowing_tokens=True
        )

    # Применяем токенизацию к датасету
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"]
    )

    # Загружаем модель для masked language modeling
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")

    # Создаем data collator для случайного маскирования токенов (15% токенов)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    # Определяем аргументы обучения
    training_args = TrainingArguments(
        output_dir="./bert_mlm",
        overwrite_output_dir=True,
        num_train_epochs=1,
        per_device_train_batch_size=8,  # Уменьшаем размер батча для экономии памяти
        save_steps=10000,
        save_total_limit=2,
        prediction_loss_only=True,
        logging_steps=500,
    )

    # Создаем объект Trainer для обучения модели
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    # Запускаем процесс обучения
    trainer.train()

    # Сохраняем модель и токенизатор
    trainer.save_model("./bert_mlm")
    tokenizer.save_pretrained("./bert_mlm")

    return model, tokenizer

def fill_placeholders_and_track(text, fill_mask, tokenizer):
    """
    Функция ищет шаблоны вида *[число]* и последовательно заменяет их
    на предсказанные моделью слова. Возвращает список заполненных слов.
    """
    # Регулярное выражение для поиска шаблона, например: *[0]*, *[1]* и т.д.
    pattern = r"\*\[\d+\]\*"

    # Для хранения заполненных слов
    filled_words = []

    # Пока в тексте встречается хотя бы один шаблон
    while re.search(pattern, text):
        # Находим первое вхождение шаблона
        placeholder = re.search(pattern, text).group(0)

        # Заменяем первое найденное вхождение на токен [MASK]
        text = re.sub(pattern, tokenizer.mask_token, text, count=1)

        # Проверяем длину текста и разбиваем на части, если необходимо
        if len(tokenizer.encode(text)) > 512:
            # Находим позицию маски
            mask_pos = text.find(tokenizer.mask_token)

            # Вырезаем кусок текста вокруг маски, не превышающий 512 токенов
            start = max(0, mask_pos - 200)
            end = min(len(text), mask_pos + 200)
            text_chunk = text[start:end]

            # Получаем предсказание для токена [MASK] в обрезанном тексте
            predictions = fill_mask(text_chunk)
            predicted_token = predictions[0]['token_str'].strip()

            # Сохраняем предсказанное слово и его позицию в шаблоне
            filled_words.append((placeholder, predicted_token))

            # Заменяем маску в оригинальном тексте
            text = text[:mask_pos] + predicted_token + text[mask_pos + len(tokenizer.mask_token):]
        else:
            # Получаем предсказание для токена [MASK]
            predictions = fill_mask(text)
            predicted_token = predictions[0]['token_str'].strip()

            # Сохраняем предсказанное слово и его позицию в шаблоне
            filled_words.append((placeholder, predicted_token))

            # Заменяем первый найденный [MASK] на предсказанное слово
            text = text.replace(tokenizer.mask_token, predicted_token, 1)

    return text, filled_words

def validate_model(model, tokenizer):
    """
    Функция для валидации: загружает test.txt, заполняет в нем шаблоны
    и выводит список заполненных слов.
    """
    # Загружаем текст для валидации
    with open("test.txt", "r", encoding="utf-8") as f:
        test_text = f.read()

    # Создаем pipeline для задачи fill-mask
    fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

    # Заполняем все шаблоны в тексте и получаем список заполненных слов
    filled_text, filled_words = fill_placeholders_and_track(test_text, fill_mask, tokenizer)

    print("Заполненный текст:\n")
    print(filled_text)

    print("\nСписок заполненных слов:\n")
    for placeholder, word in filled_words:
        print(f"{placeholder} → {word}")

    # Если нужен только список слов без плейсхолдеров
    print("\nТолько заполненные слова:\n")
    for _, word in filled_words:
        print(word)


def main():
    # Обучаем модель на train.txt
    model, tokenizer = train_model()
    # Валидируем модель на test.txt
    validate_model(model, tokenizer)

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss


Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (15707 > 512). Running this sequence through the model will result in indexing errors


Заполненный текст:

But I must think, he thought. Because that is all I have left. That and more. I wonder
how the great man would have liked the way I hit him in the brain? It was no great
thing, he thought. Any man could do it. But do you think my hands were as great a handicap
as the bone spurs? I cannot know. I never had anything wrong with my heel except the
time the sting ray stung it when I stepped on him while swimming and paralyzed the lower
jaw and made the unbearable pain.
"Think about something cheerful, old man," he said. "Every minute now you are closer
to home. You sail lighter for the loss of forty pounds."
He knew quite well the pattern of what could happen when he reached the inner part
of the current. But there was nothing to be done now.
"Yes there is," he said aloud. "I can lash my knife to the butt of one of the oars."
So he did that with the tiller under his arm and the sheet of the sail under his foot.
"Now," he said. "I am still an old man. But I am not unarmed

# НОРМ КОД


In [23]:
import re
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import load_dataset

In [24]:
# Загружаем датасет из текстового файла
dataset = load_dataset("text", data_files={"train": "train.txt"}, split="train")

# Загружаем предобученный токенизатор BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Функция для токенизации и разбиения длинных текстов на части
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        stride=128,  # Перекрытие между частями текста
        return_overflowing_tokens=True
    )

# Применяем токенизацию к датасету
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# Загружаем модель для masked language modeling
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Создаем data collator для случайного маскирования токенов (15% токенов)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Определяем аргументы обучения
training_args = TrainingArguments(
    output_dir="./bert_mlm",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,  # Уменьшаем размер батча для экономии памяти
    save_steps=10000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=500,
)

# Создаем объект Trainer для обучения модели
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# Запускаем процесс обучения
trainer.train()

Map:   0%|          | 0/3711 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss


TrainOutput(global_step=464, training_loss=2.6174779431573274, metrics={'train_runtime': 397.3741, 'train_samples_per_second': 9.339, 'train_steps_per_second': 1.168, 'total_flos': 976753060300800.0, 'train_loss': 2.6174779431573274, 'epoch': 1.0})

In [25]:
# Сохраняем модель и токенизатор
trainer.save_model("./bert_mlm")
tokenizer.save_pretrained("./bert_mlm")

# Создаем pipeline для задачи fill-mask
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [26]:
# Загружаем текст для валидации
with open("test.txt", "r", encoding="utf-8") as f:
    test_text = f.read()

# Регулярное выражение для поиска шаблона, например: *[0]*, *[1]* и т.д.
pattern = r"\*\[\d+\]\*"

# Для хранения заполненных слов
filled_words = []

# Пока в тексте встречается хотя бы один шаблон
while re.search(pattern, test_text):
    # Находим первое вхождение шаблона
    placeholder = re.search(pattern, test_text).group(0)

    # Заменяем первое найденное вхождение на токен [MASK]
    test_text = re.sub(pattern, tokenizer.mask_token, test_text, count=1)

    # Проверяем длину текста и разбиваем на части, если необходимо
    if len(tokenizer.encode(test_text)) > 512:
        # Находим позицию маски
        mask_pos = test_text.find(tokenizer.mask_token)

        # Вырезаем кусок текста вокруг маски, не превышающий 512 токенов
        start = max(0, mask_pos - 200)
        end = min(len(test_text), mask_pos + 200)
        text_chunk = test_text[start:end]

        # Получаем предсказание для токена [MASK] в обрезанном тексте
        predictions = fill_mask(text_chunk)
        predicted_token = predictions[0]['token_str'].strip()

        # Сохраняем предсказанное слово и его позицию в шаблоне
        filled_words.append((placeholder, predicted_token))

        # Заменяем маску в оригинальном тексте
        test_text = test_text[:mask_pos] + predicted_token + test_text[mask_pos + len(tokenizer.mask_token):]
    else:
        # Получаем предсказание для токена [MASK]
        predictions = fill_mask(test_text)
        predicted_token = predictions[0]['token_str'].strip()

        # Сохраняем предсказанное слово и его позицию в шаблоне
        filled_words.append((placeholder, predicted_token))

        # Заменяем первый найденный [MASK] на предсказанное слово
        test_text = test_text.replace(tokenizer.mask_token, predicted_token, 1)

Token indices sequence length is longer than the specified maximum sequence length for this model (15707 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import pandas as pd
# Вывод результатов
print("Заполненный текст:\n")
print(test_text)

print("\nСписок заполненных слов:\n")
for placeholder, word in filled_words:
    print(f"{placeholder} → {word}")

print("\nТолько заполненные слова:\n")
df = pd.DataFrame()
lst = []
for _, word in filled_words:
    lst.append(word)

In [36]:
lst2 = []
for i in range(len(lst)):
  lst2.append(i)

In [37]:
df['id'] = lst2
df['target'] = lst

In [38]:
df

,id,target
0,0,that
1,1,more
2,2,man
3,3,do
4,4,the
...,...,...
546,546,did
547,547,the
548,548,do
549,549,here
